# Yoga Pose Detection Training and Analysis

This notebook helps you train and improve your yoga pose detection model using MediaPipe landmarks.

## 1. Setup and Imports

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import joblib
from datetime import datetime

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All imports successful!")

## 2. Load and Analyze Training Data

In [ ]:
def load_training_data(data_dir="training_data"):
    """Load training data from collected samples"""
    landmarks_dir = os.path.join(data_dir, "landmarks")
    
    if not os.path.exists(landmarks_dir):
        print(f"❌ Training data directory not found: {landmarks_dir}")
        print("💡 Run collect_training_data.py first to collect samples")
        return None
    
    data = []
    
    for filename in os.listdir(landmarks_dir):
        if filename.endswith('.json'):
            filepath = os.path.join(landmarks_dir, filename)
            
            with open(filepath, 'r') as f:
                sample = json.load(f)
                data.append(sample)
    
    print(f"📊 Loaded {len(data)} training samples")
    return data

# Load the data
training_data = load_training_data()

if training_data:
    # Analyze the dataset
    poses = [sample['pose_name'] for sample in training_data]
    pose_counts = pd.Series(poses).value_counts()
    
    print("\n📈 Dataset Overview:")
    print(pose_counts)
    
    # Plot distribution
    plt.figure(figsize=(10, 6))
    pose_counts.plot(kind='bar')
    plt.title('Training Samples per Pose')
    plt.xlabel('Pose')
    plt.ylabel('Number of Samples')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 3. Feature Engineering

In [ ]:
def extract_features(landmarks):
    """Extract features from MediaPipe landmarks"""
    features = []
    
    # Basic landmark coordinates (normalized)
    for lm in landmarks:
        features.extend([lm['x'], lm['y'], lm['z'], lm['visibility']])
    
    return features

def calculate_pose_angles(landmarks):
    """Calculate key angles from landmarks"""
    angles = {}
    
    def calc_angle(p1, p2, p3):
        """Calculate angle between three points"""
        a = np.array([p1['x'], p1['y']])
        b = np.array([p2['x'], p2['y']])
        c = np.array([p3['x'], p3['y']])
        
        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians * 180.0 / np.pi)
        
        if angle > 180.0:
            angle = 360 - angle
        
        return angle
    
    # Key body angles
    try:
        # Knee angles
        angles['left_knee'] = calc_angle(landmarks[23], landmarks[25], landmarks[27])  # Hip-Knee-Ankle
        angles['right_knee'] = calc_angle(landmarks[24], landmarks[26], landmarks[28])
        
        # Elbow angles
        angles['left_elbow'] = calc_angle(landmarks[11], landmarks[13], landmarks[15])  # Shoulder-Elbow-Wrist
        angles['right_elbow'] = calc_angle(landmarks[12], landmarks[14], landmarks[16])
        
        # Hip angles
        angles['left_hip'] = calc_angle(landmarks[11], landmarks[23], landmarks[25])  # Shoulder-Hip-Knee
        angles['right_hip'] = calc_angle(landmarks[12], landmarks[24], landmarks[26])
        
        # Shoulder angles
        angles['left_shoulder'] = calc_angle(landmarks[13], landmarks[11], landmarks[23])  # Elbow-Shoulder-Hip
        angles['right_shoulder'] = calc_angle(landmarks[14], landmarks[12], landmarks[24])
        
    except Exception as e:
        print(f"Error calculating angles: {e}")
    
    return angles

def create_feature_dataset(training_data):
    """Create feature dataset for machine learning"""
    features = []
    labels = []
    
    for sample in training_data:
        landmarks = sample['landmarks']
        pose_name = sample['pose_name']
        
        # Extract basic features
        sample_features = extract_features(landmarks)
        
        # Add angle features
        angles = calculate_pose_angles(landmarks)
        sample_features.extend(list(angles.values()))
        
        features.append(sample_features)
        labels.append(pose_name)
    
    return np.array(features), np.array(labels)

if training_data:
    # Create feature dataset
    X, y = create_feature_dataset(training_data)
    print(f"\n🎯 Feature dataset created:")
    print(f"   Features shape: {X.shape}")
    print(f"   Labels shape: {y.shape}")
    print(f"   Unique poses: {np.unique(y)}")

## 4. Train Pose Classification Model

In [ ]:
if training_data and len(training_data) > 10:  # Need minimum samples
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train Random Forest classifier
    rf_model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        class_weight='balanced'
    )
    
    rf_model.fit(X_train_scaled, y_train)
    
    # Evaluate model
    train_score = rf_model.score(X_train_scaled, y_train)
    test_score = rf_model.score(X_test_scaled, y_test)
    
    print(f"\n🎯 Model Performance:")
    print(f"   Training accuracy: {train_score:.3f}")
    print(f"   Test accuracy: {test_score:.3f}")
    
    # Detailed classification report
    y_pred = rf_model.predict(X_test_scaled)
    print("\n📊 Classification Report:")
    print(classification_report(y_test, y_pred))
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=rf_model.classes_, 
                yticklabels=rf_model.classes_)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.tight_layout()
    plt.show()
    
    # Feature importance
    feature_importance = rf_model.feature_importances_
    
    # Save the model
    model_dir = "models"
    os.makedirs(model_dir, exist_ok=True)
    
    joblib.dump(rf_model, f"{model_dir}/pose_classifier.pkl")
    joblib.dump(scaler, f"{model_dir}/feature_scaler.pkl")
    
    print(f"\n💾 Model saved to {model_dir}/")
    
else:
    print("❌ Not enough training data. Collect at least 10 samples per pose.")

## 5. Real-time Pose Detection Test

In [ ]:
def test_realtime_detection():
    """Test the trained model with real-time webcam input"""
    
    # Load trained model
    try:
        model = joblib.load("models/pose_classifier.pkl")
        scaler = joblib.load("models/feature_scaler.pkl")
        print("✅ Loaded trained model")
    except:
        print("❌ No trained model found. Train the model first.")
        return
    
    # Initialize MediaPipe
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    
    pose = mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        smooth_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )
    
    cap = cv2.VideoCapture(0)
    
    print("🎥 Starting real-time pose detection...")
    print("Press 'q' to quit")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Mirror frame
        frame = cv2.flip(frame, 1)
        
        # Convert BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb.flags.writeable = False
        
        # Process with MediaPipe
        results = pose.process(frame_rgb)
        
        # Convert back to BGR
        frame_rgb.flags.writeable = True
        frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
        
        if results.pose_landmarks:
            # Draw landmarks
            mp_drawing.draw_landmarks(
                frame_bgr,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2)
            )
            
            # Extract features for prediction
            landmarks = results.pose_landmarks.landmark
            landmark_data = []
            
            for lm in landmarks:
                landmark_data.append({
                    'x': lm.x, 'y': lm.y, 'z': lm.z, 'visibility': lm.visibility
                })
            
            # Extract features
            features = extract_features(landmark_data)
            angles = calculate_pose_angles(landmark_data)
            features.extend(list(angles.values()))
            
            # Predict pose
            features_scaled = scaler.transform([features])
            prediction = model.predict(features_scaled)[0]
            confidence = np.max(model.predict_proba(features_scaled))
            
            # Display prediction
            cv2.putText(frame_bgr, f"Pose: {prediction}", (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame_bgr, f"Confidence: {confidence:.2f}", (10, 70), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.imshow('Pose Detection Test', frame_bgr)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Uncomment to test real-time detection
# test_realtime_detection()

## 6. Generate Improved Pose Detection Rules

In [ ]:
def analyze_pose_characteristics(training_data):
    """Analyze characteristics of each pose for better rule-based detection"""
    
    pose_stats = {}
    
    for sample in training_data:
        pose_name = sample['pose_name']
        landmarks = sample['landmarks']
        
        if pose_name not in pose_stats:
            pose_stats[pose_name] = {
                'angles': [],
                'samples': 0
            }
        
        # Calculate angles for this sample
        angles = calculate_pose_angles(landmarks)
        pose_stats[pose_name]['angles'].append(angles)
        pose_stats[pose_name]['samples'] += 1
    
    # Calculate statistics for each pose
    pose_rules = {}
    
    for pose_name, stats in pose_stats.items():
        if stats['samples'] < 3:  # Need minimum samples
            continue
            
        angles_df = pd.DataFrame(stats['angles'])
        
        pose_rules[pose_name] = {
            'mean_angles': angles_df.mean().to_dict(),
            'std_angles': angles_df.std().to_dict(),
            'min_angles': angles_df.min().to_dict(),
            'max_angles': angles_df.max().to_dict(),
            'samples': stats['samples']
        }
    
    return pose_rules

def generate_detection_rules(pose_rules):
    """Generate improved detection rules based on training data"""
    
    detection_code = """
# Auto-generated pose detection rules based on training data
# Generated on: {}

POSE_DETECTION_RULES = {{
""".format(datetime.now().isoformat())
    
    for pose_name, rules in pose_rules.items():
        detection_code += f'    "{pose_name}": {{\n'
        detection_code += f'        "name": "{pose_name.replace("_", " ").title()}",\n'
        detection_code += f'        "samples_used": {rules["samples"]},\n'
        detection_code += f'        "ideal_angles": {{\n'
        
        for angle_name, mean_val in rules['mean_angles'].items():
            std_val = rules['std_angles'].get(angle_name, 10)
            detection_code += f'            "{angle_name}": {mean_val:.1f},  # ±{std_val:.1f}\n'
        
        detection_code += f'        }},\n'
        detection_code += f'        "tolerance_ranges": {{\n'
        
        for angle_name, std_val in rules['std_angles'].items():
            tolerance = max(10, std_val * 2)  # At least 10 degrees tolerance
            detection_code += f'            "{angle_name}": {tolerance:.1f},\n'
        
        detection_code += f'        }}\n'
        detection_code += f'    }},\n'
    
    detection_code += "}\n"
    
    return detection_code

if training_data:
    # Analyze pose characteristics
    pose_rules = analyze_pose_characteristics(training_data)
    
    print("📊 Pose Analysis Results:")
    for pose_name, rules in pose_rules.items():
        print(f"\n{pose_name} ({rules['samples']} samples):")
        for angle_name, mean_val in rules['mean_angles'].items():
            std_val = rules['std_angles'].get(angle_name, 0)
            print(f"  {angle_name}: {mean_val:.1f}° ±{std_val:.1f}°")
    
    # Generate improved detection rules
    detection_code = generate_detection_rules(pose_rules)
    
    # Save to file
    with open("improved_pose_rules.py", "w") as f:
        f.write(detection_code)
    
    print("\n💾 Improved detection rules saved to 'improved_pose_rules.py'")
    print("🔄 You can now integrate these rules into your pose_detector.py")

## 7. Next Steps

1. **Collect More Data**: Use `collect_training_data.py` to gather more samples
2. **Improve Features**: Add more sophisticated angle calculations and body ratios
3. **Try Different Models**: Experiment with SVM, Neural Networks, etc.
4. **Real-time Integration**: Integrate the trained model into your main application
5. **Continuous Learning**: Set up a system to continuously improve with user feedback